In [1]:
import pandas as pd


In [2]:
df_UserItems=pd.read_csv('user_items_cleaned.csv')
df_SteamGames=pd.read_csv('steam_games_cleaned.csv')
df_UserReviews=pd.read_csv('user_reviews_cleaned.csv')

In [6]:

def PlayTimeGenre(genero):
    # Se filtra el DataFrame original para obtener solo las columnas relevantes
    game_columns = ['item_id', 'name', 'year', 'Indie', 'Action', 'Casual', 'Adventure', 'Strategy',
                    'Simulation', 'RPG', 'Free to Play', 'Early Access', 'Sports',
                    'Massively Multiplayer', 'Racing', 'Design &amp; Illustration', 'Utilities']

    # Se crea un nuevo DataFrame llamado df_games que contiene solo las columnas especificadas en game_columns.
    df_games = df_SteamGames[game_columns]

    # Se hace un join entre df_games y df_UserItems utilizando 'item_id' como clave
    df_joined = pd.merge(df_games, df_UserItems[['item_id', 'playtime_forever']], on='item_id', how='inner')

    # Se filtra el DataFrame resultante para el género específico
    genre_df = df_joined[df_joined[genero] == 1]

    # Se calcula la suma de horas jugadas por año y se determina el año con más horas jugadas
    total_hours_by_year = genre_df.groupby('year')['playtime_forever'].sum()
    max_year = total_hours_by_year.idxmax()

    # Se retorna el resultado en un formato específico
    result = {"Año de lanzamiento con más horas jugadas para Genero {}: {}".format(genero, max_year)}
    return result



{'Año de lanzamiento con más horas jugadas para Indie: 2006'}


In [8]:
# Ejemplo de uso:
genero_a_buscar = 'Casual'  
resultado = PlayTimeGenre(genero_a_buscar)
print(resultado)

{'Año de lanzamiento con más horas jugadas para Casual: 2013'}


In [20]:
def UserForGenre(genero):
    # Se filtra el DataFrame original para obtener solo las columnas relevantes
    game_columns = ['item_id', 'name', 'year', 'Indie', 'Action', 'Casual', 'Adventure', 'Strategy',
                    'Simulation', 'RPG', 'Free to Play', 'Early Access', 'Sports',
                    'Massively Multiplayer', 'Racing', 'Design &amp; Illustration', 'Utilities']
    
    # Se crea un nuevo DataFrame llamado df_games que contiene solo las columnas especificadas en game_columns.
    df_games = df_SteamGames[game_columns]
    
    # Se hace un join entre df_games y df_UserItems utilizando 'item_id' como clave
    df_joined = pd.merge(df_games, df_UserItems[['item_id', 'playtime_forever','user_id']], on='item_id', how='inner')

    # Se filtra el DataFrame resultante para el género específico
    genre_df = df_joined[df_joined[genero] == 1]

    # Se agrupa por usuario y año, sumamos las horas jugadas y se encuentra el usuario con más horas jugadas para el género dado
    total_hours_by_user_and_year = genre_df.groupby(['user_id', 'year'])['playtime_forever'].sum()
    max_user = total_hours_by_user_and_year.groupby('user_id').sum().idxmax()


    # Se obtiene la lista de acumulación de horas jugadas por año para el usuario con más horas jugadas
    max_user_hours_by_year = total_hours_by_user_and_year.loc[max_user].reset_index()
    max_user_hours_list = [{"Año": int(row['year']), "Horas": row['playtime_forever']} for _, row in max_user_hours_by_year.iterrows()]

    # Se retorna el resultado en un formato específico
    result = {"Usuario con más horas jugadas para Género {}".format(genero): max_user, "Horas jugadas": max_user_hours_list}
    return result


In [21]:
# Ejemplo de uso:
genero_a_buscar = 'Casual'  
resultado = UserForGenre(genero_a_buscar)
print(resultado)

{'Usuario con más horas jugadas para Género Casual': 'REBAS_AS_F-T', 'Horas jugadas': [{'Año': 2001, 'Horas': 0.18}, {'Año': 2007, 'Horas': 7.1}, {'Año': 2008, 'Horas': 22.5}, {'Año': 2009, 'Horas': 124.79}, {'Año': 2010, 'Horas': 845.0699999999999}, {'Año': 2011, 'Horas': 691.41}, {'Año': 2012, 'Horas': 1278.84}, {'Año': 2013, 'Horas': 655.63}, {'Año': 2014, 'Horas': 1527.92}, {'Año': 2015, 'Horas': 4094.92}, {'Año': 2016, 'Horas': 5627.18}, {'Año': 2017, 'Horas': 38.67}]}


In [8]:
def UsersRecommend(año):
    
    # Se filtra el DataFrame resultante para el año específico
    reduce_df = df_UserReviews[(df_UserReviews['year'] == año) & 
                               (df_UserReviews['recommend'] == True) & 
                               (df_UserReviews['sentiment_analysis'].isin([1,2]))]
    
    # Se hace un join entre df_UserReviews y df_games utilizando 'item_id' como clave
    merged_reviews = pd.merge(reduce_df, df_SteamGames[['item_id', 'name']], on='item_id', how='left')
   
    # Se calcula la frecuencia de cada juego
    game_count= merged_reviews['name'].value_counts()

    
    # Se selecciona el top 3 de juegos más recomendados, estos estan ordenados de mayor a menor por el value_counts
    top_3_games = game_count.head(3)

    # Se retorna el resultado en un formato específico
    result = [{"Puesto {}".format(i+1): name} for i, (name, _) in enumerate(top_3_games.items())]
    
    return result



In [9]:
# Ejemplo de uso:
año = 2011
resultado = UsersRecommend(año)
print(resultado)

[{'Puesto 1': 'Team Fortress 2'}, {'Puesto 2': 'Portal 2'}, {'Puesto 3': 'Terraria'}]


In [63]:
def UsersWorstDeveloper(año):
    # Se filtra el DataFrame resultante para el año específico
    reduce_df = df_UserReviews[(df_UserReviews['year'] == año) & 
                               (df_UserReviews['recommend'] == False) & 
                               (df_UserReviews['sentiment_analysis'].isin([0, -1]))]
    
    # Se hace un join entre df_UserReviews y df_steamgames utilizando 'item_id' como clave
    merged_reviews = pd.merge(reduce_df, df_SteamGames[['item_id', 'name','developer']], on='item_id', how='left')
   
    # Se calcula la frecuencia de cada juego
    game_count= merged_reviews['developer'].value_counts()

    
    # Se selecciona el top 3 de juegos más recomendados, estos estan ordenados de mayor a menor por el value_counts
    top_3_worst_games = game_count.head(3)

    # Se retorna el resultado en un formato específico
    result = [{"Puesto {}".format(i+1): developer} for i, (developer, _) in enumerate(top_3_worst_games.items())]
    
    return result


In [65]:
# Ejemplo de uso:
año = 2015
resultado = UsersWorstDeveloper(año)
print(resultado)

[{'Puesto 1': 'Valve'}, {'Puesto 2': 'Bohemia Interactive'}, {'Puesto 3': 'Facepunch Studios'}]


In [59]:
def sentiment_analysis(desarrolladora):

    # Se hace un join entre df_UserReviews y df_games utilizando 'item_id' como clave
    merged_reviews = pd.merge(df_UserReviews,df_SteamGames[['item_id','developer']], on='item_id', how='left')

    # Se filtra el DataFrame de reseñas por la desarrolladora específica
    reviews_by_developer = merged_reviews[merged_reviews['developer'] == desarrolladora]

    # Se realiza el análisis de sentimiento
    sentiment_counts = reviews_by_developer['sentiment_analysis'].value_counts()

    # Se crea el diccionario de resultado
    result = {desarrolladora: {
        'Negative': sentiment_counts.get(0, 0),
        'Neutral': sentiment_counts.get(1, 0),
        'Positive': sentiment_counts.get(2, 0)
    }}
    return result


In [67]:
# Ejemplo de uso:
desarrolladora = 'Valve'
resultado = sentiment_analysis(desarrolladora)
print(resultado)

{'Valve': {'Negative': 911, 'Neutral': 779, 'Positive': 4710}}
